In [3]:
from sklearn.externals import joblib
import json as js
import os
import sqlite3 as db


class clTree():
    
    def __init__(self,namefile):
        self.namefile = namefile
        self.route = '/home/jhonex/Documentos/codigo/codigo_py_soport/tre_editor_versions/tree_editor_local/resource_bd/bd_script/resource_dbtree.sql'
        self.tree_config = {'initial_state': '','templates': [],'max_attempts': 0,
            'topics':{'pendiente':'pendiente'},'transitions':[],'models': [],'final_state' : ''}
        self.insertItemTemplate('')
        self.validateStatedata('')
        self.makeListPkl('')
        self.searchmodel('','')
        self.validateTemplate('','','','')
        self.validatetrasition('','','','')
        
    def getTreeConfig(self):
        return self.tree_config
    
    def makeListState(self,state):
        if(self.namefile != '' and state != ''):
            msj = ''
            if(self.validateStatedata(state)):
                msj = '!La clave no se pudo crear ya exite¡'
            else:
                cn = db.connect(self.route)
                cur = cn.cursor()
                cur.execute("insert into tbl_listState (nameFile,keylistState) values ('%s','%s')"%(self.namefile,state))
                cn.commit()
                cur.close()
                msj = '!clave creada¡'
        return msj   
    
    def validateStatedata(self,state):
        li_1 = []
        cn = db.connect(self.route)
        cur = cn.cursor()
        cur.execute('SELECT keylistState FROM tbl_listState where nameFile="%s" and keylistState="%s"'%(self.namefile,state))
        sc_list = cur.fetchall()
        cn.commit()
        cur.close()
        return sc_list
    
    def getState(self):
        li_1 = []
        cn = db.connect(self.route)
        cur = cn.cursor()
        cur.execute("SELECT keylistState FROM tbl_listState where nameFile='%s'"%(self.namefile))
        sc_list = cur.fetchall()
        cn.commit()
        cur.close()
        if(sc_list):
            sc_list = [list(i) for i in sc_list]
            for i in sc_list:
                li_1.append(i[0])
            sc_list = li_1
        else:
            sc_list = '!no existen estados asociados para este archivo¡'
        return  sc_list
    
    def deleteState(self,state):
        cn = db.connect(self.route)
        cur = cn.cursor()
        cur.execute('DELETE FROM tbl_listState WHERE keylistState="%s" AND nameFile="%s"'%(state,self.namefile))
        cn.commit()
        cur.close()
        msj = '!Eliminado¡'
        return msj
    
    def editState(self,state,modState):
        cn = db.connect(self.route)
        cur = cn.cursor()
        cur.execute('UPDATE tbl_listState SET keylistState="%s" WHERE nameFile="%s" AND keylistState="%s"'%(modState, self.namefile,state ))
        cn.commit()
        cur.close()
        msj = '!Editado¡'
        return msj
    
    def makeListTemplates(self,fileTemplate):
        if(fileTemplate):   
            try:
                file =  json.load(open(fileTemplate,'r'))
                file = list(file.keys())
                msj = self.insertItemTemplate(file)
            except OSError:
                msj = '!No se pudo cargar archivo¡'
        else:
            msj = '!Ingresa un Archivo¡'
        return msj
    
    def insertItemTemplate(self,lis):
        if(lis):
            for row in lis:
                cn = db.connect(self.route)
                cur = cn.cursor()
                cur.execute('delete from tbl_keyslabel where nameFile="%s" and keylabel="%s"'%(self.namefile,row))
                cur.execute('INSERT INTO tbl_keyslabel (nameFile ,keylabel) VALUES ("%s","%s")'%(self.namefile,row))
                cn.commit()
                cur.close()
                msj = '!Lista creada¡'
        else:
            msj = ''
        return  msj
    
    def getListKeyLabel(self):
        li_1 = []
        cn = db.connect(self.route)
        cur = cn.cursor()
        cur.execute("SELECT keylabel FROM tbl_keyslabel where nameFile='%s'"%(self.namefile))
        sc_list = cur.fetchall()
        cn.commit()
        cur.close()
        if(sc_list):
            sc_list = [list(i) for i in sc_list] 
            for i in sc_list:
                li_1.append(i[0])
            
            sc_list = li_1
        else:
            sc_list = '!no existen estados asociados para este archivo¡'
        return sc_list
    
    def makeListPkl(self,filepkl):
        if(filepkl != ''):
            models = joblib.load(filepkl)
        else:
            models = ''
        return list(models)
    
    def loadPKL(self,namedoc):
        msj = ''
        if(namedoc):
            namedoc = self.makeListPkl(namedoc)
            for row in namedoc:
                cn = db.connect(self.route)
                cur = cn.cursor()
                cur.execute('delete from tbl_LabelPkl where nameFile="%s" and keyLabelPkl="%s"'%(self.namefile,row))
                cur.execute('INSERT INTO tbl_LabelPkl (nameFile, keyLabelPkl) VALUES ("%s","%s")'%(self.namefile,row))
                cn.commit()
                cur.close()
            msj = 'quedo cargado'
        else:
            msj = 'no se pudo cargar datos'
        return msj
    
    def deleteAllContent(self):
        cn = db.connect(self.route)
        cur = cn.cursor()
        cur.execute('delete from tbl_keyslabel;')
        cur.execute('delete from tbl_listState;')
        cur.execute('delete from tbl_LabelPkl;')
        cn.commit()
        cur.close()
    
    def getloadPKL(self):
        li_1 = []
        cn = db.connect(self.route)
        cur = cn.cursor()
        cur.execute("SELECT keyLabelPkl FROM tbl_LabelPkl where nameFile='%s'"%(self.namefile))
        sc_list = cur.fetchall()
        cn.commit()
        cur.close()
        if(sc_list):
            sc_list = [list(i) for i in sc_list]
            for i in sc_list:
                li_1.append(i[0])
            sc_list = li_1
        else:
            sc_list = '!no existen estados asociados para este archivo¡'
        return  sc_list
      
    def searchmodel(self,i_d,state):
        flag = True 
        if(i_d != '' and state  != '' ):     

            try:    
                model = self.tree_config['models']
                for i in  model:
                    if(i['id'] == i_d and i['state'] == state):
                        flag = False
            except KeyError as err:
                print('el archivo cargado no tiene la estructura correcta')
        return flag
    
    def makemodel(self,i_d,state):
        if(self.searchmodel(i_d,state)):
            self.tree_config['models'].append({'id': i_d, 'state': state})
            msj = 'Exito al agregar modelo'
        else:
            msj = 'Error al crear modelo'
        return msj
    
    def loadTreeConfig(self):
        resource = {"final_state":0,}
        if(self.namefile):   
            try:
                try:
            
                    file =  json.load(open(self.namefile,'r'))
                    self.tree_config['templates'] = file['templates']
                    self.tree_config['topics'] = file['topics']
                    self.tree_config['models'] = file['templates']
                    self.tree_config['transitions'] = file['transitions']
                    self.tree_config['initial_state'] = file['initial_state']
                    self.tree_config['final_state'] = file['final_state']
                    self.tree_config['max_attempts'] = file['max_attempts']
                    msj = '!Recursos cargados¡'
                except KeyError as err:
                      msj = '!No se pudo cargar archivo\n la estructura del archivo no es compatible¡'
            
            except OSError:
                msj = '!No se pudo cargar archivo, Archivo inexistente¡'
        else:
            msj = '!Ingresa un Archivo¡'
        return msj    
    
    def maketemplate(self,i_d,label,state,topic):
        msj = ''
        if(self.validateTemplate(i_d,label,state,topic)):
            self.tree_config['templates'].append({'id': i_d,'label': label,'state': state,'topic': topic})
            msj = 'Template agregado'
        else: 
            msj = 'Error al agregar template'
        
        return msj

    
    def validateTemplate(self,i_d,label,state,topic):
        flag = True 
        if(i_d != '' and state  != '' and label != '' ):     

            try:    
                model = self.tree_config['templates']
                for i in  model:
                    if(i['id'] == i_d and i['state'] == state and i['label'] == label and i['topic'] == topic):
                        flag = False
            except KeyError as err:
                print('el archivo cargado no tiene la estructura correcta')
        return flag
    
    def validatetrasition(self,endstate, is_forboo,label,ini_state):
        flag = True 
        if(endstate != '' and is_forboo != '' and label != '' and ini_state != ''):    
            try:    
                model = self.tree_config['transitions']
                for i in  model:
                    if(i['ending_state'] == endstate and i['is_forced'] == is_forboo and i['label'] == label and i['starting_state'] == ini_state):
                        flag = False
            except KeyError as err:
                print('el archivo cargado no tiene la estructura correcta')
        return flag
    
    def maketrasition(self,endstate, is_forboo,label,ini_state):
        if(self.validatetrasition(endstate, is_forboo,label,ini_state)):    
            self.tree_config['transitions'].append({'ending_state': endstate, 
                                           'is_forced': is_forboo, 
                                           'label': label,
                                           'starting_state': ini_state})
            msj = 'Transicion agregada'
        else:
            msj = 'Error al agregar template'
        return msj
    def makeFile(self,name,final_state, initial_state,max_attempts):
        self.tree_config['final_state'] = final_state
        self.tree_config['initial_state'] = initial_state
        self.tree_config['max_attempts'] = max_attempts
        if(self.tree_config != {} ):
            if(name != ''):
                file = name+'.json'
                self.name = name+'.json'
            else:
                file = self.name
        else:
            file = name+'.json'
            self.name = name+'.json'

        with open(file, 'w', encoding='utf-8') as outfile:
             json.dump(self.tree_config, outfile)
        return self.name

In [ ]:
import os

In [2]:
nameDoc = 'tree_dialog_config-graph.gv..png'
newName = 'tree_dialog_config-graph.gv..png'.replace('.gv..png','.png')
newName

'tree_dialog_config-graph.png'

In [3]:
os.rename(nameDoc, newName)

In [4]:
import os
import shutil
from os import path

In [11]:
import os
route = "/home/jhonex/Documents/Nuevo codigo/repositorio/codigo_py/tre_editor_versions/tree_editor_local/gerda/"
old_file = os.path.join(route, "tree_dialog_config-graph.png")
new_file = os.path.join(route, "tree_dialog_config-graph_gerdaaaaaa.png")
os.rename(old_file, new_file)

In [23]:
tre = {"templates": [{"state": "saludo_inicial", "cod": 0, "label": "s1", "topic": "saludo", "id": "ingreso"}, {"state": "costo_inscripcion", "cod": 1, "label": "s2", "topic": "procedimiento_primera_vez", "id": "gestion"}, {"state": "despedida", "cod": 2, "label": "s3", "topic": "despedida", "id": "despedida"}], "topics": {"none": ["s2"], "normal": ["s1"]}, "models": [{"state": "s1", "id": "ingreso", "cod": 0, "thresh": "0.5,0.5,0.5"}, {"state": "s2", "id": "gestion", "cod": 1, "thresh": "0.5,0.5,0.5"}, {"state": "s3", "id": "despedida", "cod": 2, "thresh": "0.5,0.5,0.5"}], "initial_state": "s1", "transitions": [{"cod": 0, "ending_state": "s2", "label": "pse", "is_forced": "\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t    True\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t    ", "starting_state": "s1"}, {"cod": 1, "ending_state": "s3", "label": "cuantia_indeterminada", "is_forced": "\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t    True\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t    ", "starting_state": "s2"}, {"cod": 2, "ending_state": "s1", "label": "otro", "is_forced": "\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t    True\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t    ", "starting_state": "s3"}, {"is_forced": "\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t    False\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t    ", "ending_state": "s1", "cod": 3, "label": "otro", "starting_state": "s3"}], "max_attempts": "1"}

In [12]:
bool(tre['transitions'][0]['is_forced'].replace('\t','').replace('\n','').replace(' ',''))

True

In [24]:
tre
def cleanVal(tree):
    temp1 = ''
    temp2 = ''
    lis2 = []
    for item in tree['transitions']:
        i
            temp1 = bool(item['is_forced'].replace('\t','').replace('\n','').replace(' ',''))
        
        temp2 = item
        temp2['is_forced'] = a
        lis2.append(temp2)
    tree['transitions'] = lis2
cleanVal(tre)

In [25]:
tre

{'initial_state': 's1',
 'max_attempts': '1',
 'models': [{'cod': 0,
   'id': 'ingreso',
   'state': 's1',
   'thresh': '0.5,0.5,0.5'},
  {'cod': 1, 'id': 'gestion', 'state': 's2', 'thresh': '0.5,0.5,0.5'},
  {'cod': 2, 'id': 'despedida', 'state': 's3', 'thresh': '0.5,0.5,0.5'}],
 'templates': [{'cod': 0,
   'id': 'ingreso',
   'label': 's1',
   'state': 'saludo_inicial',
   'topic': 'saludo'},
  {'cod': 1,
   'id': 'gestion',
   'label': 's2',
   'state': 'costo_inscripcion',
   'topic': 'procedimiento_primera_vez'},
  {'cod': 2,
   'id': 'despedida',
   'label': 's3',
   'state': 'despedida',
   'topic': 'despedida'}],
 'topics': {'none': ['s2'], 'normal': ['s1']},
 'transitions': [{'cod': 0,
   'ending_state': 's2',
   'is_forced': True,
   'label': 'pse',
   'starting_state': 's1'},
  {'cod': 1,
   'ending_state': 's3',
   'is_forced': True,
   'label': 'cuantia_indeterminada',
   'starting_state': 's2'},
  {'cod': 2,
   'ending_state': 's1',
   'is_forced': True,
   'label': 'otro

In [26]:
a = 'true'

In [34]:
a is not bool

True

In [ ]:
%0A%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09++++False%0A%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09%09++++

In [36]:

a = '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t    (Verdadero)\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t    '

In [38]:
a.replace('\n','').replace('\t','').replace('(','').replace(')','').replace(' ','')

'Verdadero'